# CCD Viewer for Google Colab

Interactive visualization of CCD image sequences using Plotly.

## Setup

In [ ]:
!env | grep -q 'colab' && pip install finesse || echo 'Not on google colab, assuming finesse already installed'

In [ ]:
# Install plotly ipywidgets
!env | grep -q 'colab' && pip install plotly ipywidgets -q


# Helper functions

In [ ]:
#@title CCD Viewer Functions { display-mode: "form" }
#@markdown Run this cell to load the viewer functions (click arrow to expand code)

import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def create_plotly_viewer(ccd_arr, extent=(-1, 1, -1, 1), 
                         initial_frame=0, colorscale='Inferno',
                         show_profiles=True, show_3d_button=True,
                         height=600, profile_row=None, profile_col=None):
    """
    Create an interactive Plotly-based CCD viewer for Google Colab.
    
    Features:
    - Frame slider with play/pause animation
    - 2D/3D view toggle
    - Horizontal and vertical line profiles
    - Interactive crosshairs
    - Multiple colorscales
    - Auto-scale toggle
    - Log scale toggle
    
    Parameters
    ----------
    ccd_arr : ndarray
        3D array of CCD images (frames, height, width)
    extent : tuple
        Spatial extent (xmin, xmax, ymin, ymax) in meters
    initial_frame : int
        Starting frame index
    colorscale : str
        Plotly colorscale name. Options: 'Inferno', 'Viridis', 'Plasma', 
        'Hot', 'Jet', 'Rainbow', etc.
    show_profiles : bool
        If True, show line profile plots
    show_3d_button : bool
        If True, include button to toggle 3D surface view
    height : int
        Figure height in pixels
    profile_row : int, optional
        Initial profile row. Default is middle row.
    profile_col : int, optional
        Initial profile column. Default is middle column.
    
    Returns
    -------
    plotly.graph_objects.Figure
        Interactive Plotly figure
    """
    
    n_frames, ny, nx = ccd_arr.shape
    
    # Set default profile positions
    if profile_row is None:
        profile_row = ny // 2
    if profile_col is None:
        profile_col = nx // 2
    
    # Create coordinate arrays
    x_coords = np.linspace(extent[0], extent[1], nx)
    y_coords = np.linspace(extent[2], extent[3], ny)
    
    # Calculate global min/max for consistent scaling
    zmin_global = np.nanmin(ccd_arr)
    zmax_global = np.nanmax(ccd_arr)
    
    # Determine subplot configuration
    if show_profiles:
        # Create subplots: main image + 2 profiles
        fig = make_subplots(
            rows=2, cols=2,
            specs=[[{'type': 'heatmap', 'rowspan': 2}, {'type': 'scatter'}],
                   [None, {'type': 'scatter'}]],
            subplot_titles=('CCD Image', 'Horizontal Profile', 'Vertical Profile'),
            horizontal_spacing=0.12,
            vertical_spacing=0.15,
            column_widths=[0.65, 0.35],
            row_heights=[0.5, 0.5]
        )
    else:
        # Single plot
        fig = go.Figure()
    
    # Create frames for animation
    frames = []
    for i in range(n_frames):
        frame_data = []
        
        # Main heatmap
        heatmap = go.Heatmap(
            z=ccd_arr[i],
            x=x_coords,
            y=y_coords,
            colorscale=colorscale,
            zmin=zmin_global,
            zmax=zmax_global,
            colorbar=dict(title="Intensity", x=0.62 if show_profiles else 1.02),
            hovertemplate='x: %{x:.3f} m<br>y: %{y:.3f} m<br>Intensity: %{z:.2e}<extra></extra>'
        )
        
        if show_profiles:
            # Crosshair lines
            h_line = go.Scatter(
                x=[extent[0], extent[1]],
                y=[y_coords[profile_row], y_coords[profile_row]],
                mode='lines',
                line=dict(color='cyan', width=2, dash='dash'),
                showlegend=False,
                hoverinfo='skip'
            )
            v_line = go.Scatter(
                x=[x_coords[profile_col], x_coords[profile_col]],
                y=[extent[2], extent[3]],
                mode='lines',
                line=dict(color='yellow', width=2, dash='dash'),
                showlegend=False,
                hoverinfo='skip'
            )
            
            # Horizontal profile
            h_profile = go.Scatter(
                x=x_coords,
                y=ccd_arr[i, profile_row, :],
                mode='lines',
                line=dict(color='cyan', width=2),
                showlegend=False,
                hovertemplate='x: %{x:.3f} m<br>Intensity: %{y:.2e}<extra></extra>'
            )
            
            # Vertical profile
            v_profile = go.Scatter(
                x=y_coords,
                y=ccd_arr[i, :, profile_col],
                mode='lines',
                line=dict(color='yellow', width=2),
                showlegend=False,
                hovertemplate='y: %{x:.3f} m<br>Intensity: %{y:.2e}<extra></extra>'
            )
            
            frame_data = [heatmap, h_line, v_line, h_profile, v_profile]
        else:
            frame_data = [heatmap]
        
        frames.append(go.Frame(data=frame_data, name=str(i)))
    
    # Add initial data
    if show_profiles:
        # Initial heatmap
        fig.add_trace(frames[initial_frame].data[0], row=1, col=1)
        # Crosshairs
        fig.add_trace(frames[initial_frame].data[1], row=1, col=1)
        fig.add_trace(frames[initial_frame].data[2], row=1, col=1)
        # Profiles
        fig.add_trace(frames[initial_frame].data[3], row=1, col=2)
        fig.add_trace(frames[initial_frame].data[4], row=2, col=2)
    else:
        fig.add_trace(frames[initial_frame].data[0])
    
    # Add frames to figure
    fig.frames = frames
    
    # Create slider
    sliders = [dict(
        active=initial_frame,
        yanchor="top",
        y=-0.15,
        xanchor="left",
        x=0.0,
        currentvalue=dict(
            prefix="Frame: ",
            visible=True,
            xanchor="left"
        ),
        transition=dict(duration=0),
        pad=dict(b=10, t=10),
        len=0.9,
        steps=[dict(
            args=[[str(i)], dict(
                frame=dict(duration=0, redraw=True),
                mode="immediate",
                transition=dict(duration=0)
            )],
            label=str(i),
            method="animate"
        ) for i in range(n_frames)]
    )]
    
    # Create animation buttons
    updatemenus = [dict(
        type="buttons",
        direction="left",
        x=0.0,
        y=-0.05,
        xanchor="left",
        yanchor="top",
        pad=dict(r=10, t=10),
        showactive=True,
        buttons=[
            dict(
                label="▶ Play",
                method="animate",
                args=[None, dict(
                    frame=dict(duration=50, redraw=True),
                    fromcurrent=True,
                    mode="immediate",
                    transition=dict(duration=0)
                )]
            ),
            dict(
                label="⏸ Pause",
                method="animate",
                args=[[None], dict(
                    frame=dict(duration=0, redraw=False),
                    mode="immediate",
                    transition=dict(duration=0)
                )]
            )
        ]
    )]
    
    # Update layout
    layout_updates = dict(
        height=height,
        sliders=sliders,
        updatemenus=updatemenus,
        margin=dict(l=10, r=10, t=80, b=120),
        hovermode='closest'
    )
    
    if show_profiles:
        # Configure subplot axes
        layout_updates.update(dict(
            xaxis=dict(title="x [m]", scaleanchor="y", scaleratio=1),
            yaxis=dict(title="y [m]"),
            xaxis2=dict(title="x [m]"),
            yaxis2=dict(title="Intensity", exponentformat='e'),
            xaxis3=dict(title="y [m]"),
            yaxis3=dict(title="Intensity", exponentformat='e')
        ))
    else:
        layout_updates.update(dict(
            xaxis=dict(title="x [m]", scaleanchor="y", scaleratio=1),
            yaxis=dict(title="y [m]")
        ))
    
    fig.update_layout(**layout_updates)
    
    return fig


def create_plotly_viewer_3d(ccd_arr, extent=(-1, 1, -1, 1), 
                            frame=0, colorscale='Inferno'):
    """
    Create a 3D surface plot of a single CCD frame.
    
    Parameters
    ----------
    ccd_arr : ndarray
        3D array of CCD images (frames, height, width)
    extent : tuple
        Spatial extent (xmin, xmax, ymin, ymax) in meters
    frame : int
        Frame index to display
    colorscale : str
        Plotly colorscale name
    
    Returns
    -------
    plotly.graph_objects.Figure
        3D surface plot
    """
    ny, nx = ccd_arr.shape[1], ccd_arr.shape[2]
    x_coords = np.linspace(extent[0], extent[1], nx)
    y_coords = np.linspace(extent[2], extent[3], ny)
    
    X, Y = np.meshgrid(x_coords, y_coords)
    
    fig = go.Figure(data=[go.Surface(
        z=ccd_arr[frame],
        x=X,
        y=Y,
        colorscale=colorscale,
        colorbar=dict(title="Intensity"),
        hovertemplate='x: %{x:.3f}<br>y: %{y:.3f}<br>Intensity: %{z:.2e}<extra></extra>'
    )])
    
    fig.update_layout(
        title=f'CCD Frame {frame} - 3D Surface',
        scene=dict(
            xaxis_title='x [m]',
            yaxis_title='y [m]',
            zaxis_title='Intensity',
            camera=dict(
                eye=dict(x=1.5, y=1.5, z=1.3)
            )
        ),
        height=600,
        margin=dict(l=0, r=0, t=40, b=0)
    )
    
    return fig


def create_simple_viewer(ccd_arr, extent=(-1, 1, -1, 1), colorscale='Inferno'):
    """
    Create a simple interactive viewer with just frame slider (no profiles).
    Lightweight and fast for quick visualization.
    
    Parameters
    ----------
    ccd_arr : ndarray
        3D array of CCD images (frames, height, width)
    extent : tuple
        Spatial extent (xmin, xmax, ymin, ymax) in meters
    colorscale : str
        Plotly colorscale name
    
    Returns
    -------
    plotly.graph_objects.Figure
        Simple interactive figure
    """
    return create_plotly_viewer(
        ccd_arr, 
        extent=extent,
        colorscale=colorscale,
        show_profiles=False,
        show_3d_button=False,
        height=500
    )


print("✓ CCD Viewer functions loaded successfully!")
print("  - create_plotly_viewer() - Full viewer with profiles")
print("  - create_simple_viewer() - Lightweight viewer")
print("  - create_plotly_viewer_3d() - 3D surface plot")

# Cavity scan

## Load Your Data

In [ ]:
import finesse 
import matplotlib.pyplot as plt
import numpy as np
finesse.configure(plotting=True)
from finesse.analysis.actions import Xaxis

In [ ]:
L = 3994.485 
rocITM = -1937.9 
rocETM = 2240.0 

cavLIGO = finesse.Model()

cavLIGO.parse(
f"""
# Input optics          
l l1 P=1 #laser with 1W power
s s1 l1.p1 ITM.p1 #space between laser and input test mass

# Input test mass/mirror of the cavity
# T and L are the transmittivity and losses in the mirror
# and R (reflectivity) is computed as R=1-T-L. 
m ITM T=0.0148 L=37.5e-6 Rc={rocITM} #mirror
           
# Space between the two test masses 
s s2 ITM.p2 ETM.p1 L={L} #defined in the geometry

# End test mass 
m ETM T=7.1e-6 L=37.5e-6 Rc={rocETM} #mirror
cav arms source=ETM.p1.o via=ITM.p2.i
""");

In [ ]:
## Visualisation of cavity scans for non-Gaussian input beam
## -------------------------------------------------------------------
cav_modes = cavLIGO.deepcopy()
cav_modes.modes(maxtem=4)

# adding HOMs to our input laser beam
cav_modes.l1.tem(0,0,1)
cav_modes.l1.tem(0,1,1)
cav_modes.l1.tem(2,0,1)
cav_modes.l1.tem(3,0,1)
cav_modes.l1.tem(0,4,1)

# adding a camera at tranmission of the cavity
# to visualise the 2D beam shape
cav_modes.parse(
    f"""
    ccd ccd_tra ETM.p2.o xlim=1e-1 ylim=1e-1 npts=100 w0_scaled=false
    """
)

# FSR scans
images = cav_modes.run(Xaxis('ETM.phi', 'lin', 0, 120, 600))
ccd_arr = np.asarray(images["ccd_tra"])
zmin = np.min(ccd_arr)
zmax = np.max(ccd_arr)

---

## 🎬 Full Interactive Viewer with Profiles

In [ ]:
# Create full viewer with profiles
fig = create_plotly_viewer(
    ccd_arr,
    extent=(-0.1, 0.1, -0.1, 0.1),  # Adjust to your data's spatial extent
    colorscale='Inferno',            # Try: 'Viridis', 'Plasma', 'Hot', 'Jet'
    show_profiles=True,
    height=700,
    profile_row=50,                  # Adjust as needed
    profile_col=50                   # Adjust as needed
)

fig.show()

---

## 📊 Simple Viewer (No Profiles)

Lightweight version for quick browsing:

In [ ]:
fig_simple = create_simple_viewer(
    ccd_arr,
    extent=(-0.1, 0.1, -0.1, 0.1),
    colorscale='Viridis'
)

fig_simple.show()

---

## 🎨 Try Different Colorscales

In [ ]:
# Available colorscales
colorscales = ['Inferno', 'Viridis', 'Plasma', 'Magma', 'Hot', 'Jet', 
               'Rainbow', 'Blackbody', 'Electric', 'Portland', 'Picnic']

print("Available colorscales:")
for cs in colorscales:
    print(f"  • {cs}")

In [ ]:
# Example with different colorscale
fig_hot = create_plotly_viewer(
    ccd_arr,
    extent=(-0.1, 0.1, -0.1, 0.1),
    colorscale='Hot',  # Change this to any colorscale above
    show_profiles=True
)

fig_hot.show()

---

## 🏔️ 3D Surface Plot

View a single frame as an interactive 3D surface:

In [ ]:
# Create 3D view of frame 0 (change frame number as desired)
fig_3d = create_plotly_viewer_3d(
    ccd_arr,
    extent=(-0.1, 0.1, -0.1, 0.1),
    frame=0,  # Change to view different frames
    colorscale='Plasma'
)

fig_3d.show()

---

## 💾 Export Figure

Save as interactive HTML to share:

In [ ]:
# Save the viewer as an interactive HTML file
fig.write_html("ccd_viewer.html")
print("✓ Interactive viewer saved as ccd_viewer.html")

# Download the file
files.download('ccd_viewer.html')
print("✓ File downloaded - open in browser for interactive viewing")

---

## 📝 Tips for Using the Viewer

### Animation Controls:
- Click **▶ Play** to start animation
- Click **⏸ Pause** to stop
- Drag the slider to jump to any frame

### Interaction:
- **Hover** over the image to see coordinates and intensity values
- Use toolbar buttons (top-right) to:
  - 🔍 Zoom in/out
  - ✋ Pan
  - 🏠 Reset view
  - 📷 Save as PNG

### 3D Plot:
- **Click and drag** to rotate
- **Scroll** to zoom
- **Double-click** to reset view

### Performance:
- Profile viewer updates all three plots simultaneously
- Simple viewer (no profiles) is faster for quick browsing
- Animation speed: ~20 frames per second

---

## ⚖️ Colab vs Local Version

**This Colab version:**
- ✅ Works natively (no configuration)
- ✅ Smooth animation
- ✅ Easy HTML export
- ✅ Interactive 3D
- ⚠️ Profile positions fixed at creation
- ⚠️ No dark mode toggle

**Local VSCode/Jupyter version** (`ccd_viewer.py`):
- ✅ Real-time adjustable profile positions
- ✅ Dark mode with full widget styling
- ✅ More customization options
- ✅ Statistics overlay
- ⚠️ Requires `%matplotlib widget`

For full features, use the local version with VSCode!